# Bayesian inference & machine learning example: a prior model for Australian postcodes

France's La Poste has used automated sorting since 1964.

Suppose we want to design an automated system for postcode sorting based on image recognition of postcodes.

Due to Bayes' theorem, if we had a prior model for the background frequency of different postcodes, before we even see an image of a postcode, this would improve our system's recognition accuracy.

A sample of international postcodes:
- 2000
- 3122
- 4350
- A-1220 (Vienna, Austria)
- Tsuen Wan (Hong Kong): no postcodes in HK
- 02138 (Cambridge, MA)
- EC1V 4AD (London)
- 8007 (PO boxes in Collins Street West)
- 2570 (belongs to 22 towns and suburbs around Camden, NSW, according to Wikipedia)

What prior information?

- Do all Australian postcodes have 4 digits? Yes.
- What range? 0200 to 9944
- What proportion of mail our system sees is for Australia versus overseas? 80% (assumed)
- 25% (assumed) of all mail goes to these 10 postcodes: 2000, 2001, 3000, 3001, 4000, 4001, 5000, 5001, 6000, 6001.
- What population for each postcode? (and how to handle this when e.g. Paramatta NSW has 2150 for street addresses and 2124 for postcodes). If we don't have population info by postcode, how about using a prior with state population data?
   - NSW: postcodes 1000-1999 (PO boxes), 2000-2599, 2620-2899, 2921-2999
   - ACT: 0200-0299 (PO boxes), 2600-2619, 2900-2920
   - VIC: 3000-3999, 8000-8999 (PO boxes)
   - QLD: 4000-4999, 9000-9999 (PO boxes)
   - SA: 5000-5799, 5800-5999 (PO boxes)
   - WA 6000-6797, 6800-6999 (PO boxes)
   - TAS: 7000-7799, 7800-7999 (PO boxes)
   - NT: 0800-0899, 0900-0999 (PO boxes)

### How do we encode this prior information for machine learning purposes?

In [1]:
postcodes_by_state = dict((
    ('Australian Capital Territory', set(range(2600, 2620)) | set(range(2900, 2920))),
    ('New South Wales', set(range(2000, 3000)) - set(range(2600, 2620)) - set(range(2900, 2920))),
    ('Victoria', set(range(3000, 4000))),
    ('Queensland', set(range(4000, 5000))),
    ('South Australia', set(range(5000, 5800))),
    ('Western Australia', set(range(6000, 6798))),
    ('Tasmania', set(range(7000, 7800))),
    ('Northern Territory', set(range(800, 900)))
))

### Goal: construct a prior $p(\textrm{postcode})$ over all 4-digit postcodes, where 0 represents invalid / missing / international.

We will start by using state populations as a proxy for really knowing the proportion of mail sent to each postcode. (If we obtain more data, we can update and improve our model by applying Bayes' theorem later.)

#### State populations

In [2]:
import pandas as pd

In [3]:
def fetch_state_populations():
    url = 'http://www.ausstats.abs.gov.au/Ausstats/subscriber.nsf/0/D52DEAAFCEDF7B2ACA2580EB00133359/$File/31010do001_201609.xls'

    state_populations = pd.read_excel(url, sheet_name='Table_8', skiprows=6,
                  names=['State', 'Population', '%'])

    state_populations.set_index('State', inplace=True)

    drop_row_idx = list(state_populations.index).index('Other Territories')

    state_populations.drop(state_populations.index[drop_row_idx:], inplace=True)

    state_populations['Population'] = state_populations['Population'].astype(int)
    # state_populations.to_hdf('state_populations.h5', key='populations')
    return state_populations

In [4]:
state_populations = fetch_state_populations()

In [5]:
state_populations

,Population,%
State,,
New South Wales,7757843,32.0
Victoria,6100877,25.2
Queensland,4860448,20.1
South Australia,1710804,7.1
Western Australia,2623164,10.8
Tasmania,519783,2.1
Northern Territory,245657,1.0
Australian Capital Territory,398349,1.6


These are the desired feature expectations for each state.

In [6]:
state_populations['%'].sum()

99.9

(This excludes the other territories, like Norfolk Island. Ignore this for now.)

### How to incorporate this?

... to model the probability of e.g. $p(\textrm{postcode}=3122)$?

In [7]:
def prior_state(state):
    return state_populations['%'].loc[state] / 100

In [8]:
prior_state('New South Wales')

0.32

Now we have a prior $p(\text{state})$.

### Bayes theorem:

$p(\textrm{postcode}) = \sum_{\textrm{all states}} p(\textrm{postcode | state}) p(\textrm{state})$

### Exercise:

Assuming you have a function `prior_postcode_given_state(postcode, state)`, implement this as a function `prior_postcode(postcode)`.

In [9]:
all_states = list(state_populations.index)
all_states

['New South Wales',
 'Victoria',
 'Queensland',
 'South Australia',
 'Western Australia',
 'Tasmania',
 'Northern Territory',
 'Australian Capital Territory']

In [10]:
prior_state('Australian Capital Territory')

0.016

In [11]:
def prior_postcode(postcode):
    p = 0.0
    for state in all_states:
        p += prior_postcode_given_state(postcode, state) * prior_state(state)
    assert p <= 1
    return p

### Exercise:

- Write a function `prior_postcode_given_state(postcode, state)` that assigns equal probability to each valid postcode in the corresponding state (or 0 probability for the wrong state).

In [12]:
def prior_postcode_given_state(postcode, state):
    postcodes = postcodes_by_state[state]
    return 1 / len(postcodes) if postcode in postcodes else 0

In [13]:
prior_postcode_given_state(3122, 'Victoria')

0.001

In [14]:
prior_postcode(3122)

0.000252

## Maximum entropy models: the easy way

Above we informally constructed a prior model that was as *flat* (uninformative) as possible subject to the constraint that the proportion of mail being delivered to a postcode is equal to the state's population, divided by the number of postcodes for that state.

Now we show how to derive such prior models in a more principled way using the `scikit-maxentropy` module.

In [15]:
import numpy as np
samplespace = np.arange(10000, dtype=np.uint16)

Define 0000 as "other": i.e. all international postcodes, all those addresses missing a postcode, etc.

In [16]:
samplespace

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=uint16)

In [17]:
# from maxentropy.skmaxent import MaxEntTransformer, MaxEntPrior
from maxentropy.skmaxent import FeatureTransformer, MinDivergenceModel

In [18]:
def is_valid(postcodes):
    return [200 <= postcode < 10000 for postcode in postcodes]

In [19]:
def is_nsw_street(postcodes):
    return [postcode in postcodes_by_state['New South Wales'] for postcode in postcodes]

In [20]:
def is_act_street(postcodes):
    return [postcode in postcodes_by_state['Australian Capital Territory'] for postcode in postcodes]

In [21]:
def is_vic_street(postcodes):
    return [postcode in postcodes_by_state['Victoria'] for postcode in postcodes]

In [22]:
def is_qld_street(postcodes):
    return [postcode in postcodes_by_state['Queensland'] for postcode in postcodes]

In [23]:
def is_sa_street(postcodes):
    return [postcode in postcodes_by_state['South Australia'] for postcode in postcodes]

In [24]:
def is_wa_street(postcodes):
    return [postcode in postcodes_by_state['Western Australia'] for postcode in postcodes]

In [25]:
def is_tas_street(postcodes):
    return [postcode in postcodes_by_state['Tasmania'] for postcode in postcodes]

In [26]:
def is_nt_street(postcodes):
    return [postcode in postcodes_by_state['Northern Territory'] for postcode in postcodes]

In [27]:
features = [is_valid,
            is_nsw_street, is_act_street, is_vic_street, is_qld_street,
            is_sa_street, is_wa_street, is_tas_street, is_nt_street]

In [28]:
transformer = FeatureTransformer(features, samplespace, matrix_format='ndarray', vectorized=True, verbose=False)

In [29]:
X = np.array([[2000, 3122, 90210, 8007, 5001]]).T
X

array([[ 2000],
       [ 3122],
       [90210],
       [ 8007],
       [ 5001]])

In [30]:
from sklearn.utils import check_array

In [31]:
import scipy.sparse

In [32]:
scipy.sparse.csr_matrix(X)

<5x1 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [33]:
check_array(scipy.sparse.csr_matrix(X), accept_sparse=['csr'])

<5x1 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [34]:
transformer.transform(X).A

array([[1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [35]:
samplespace

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=uint16)

In [36]:
samplespace.reshape(-1, 1)

array([[   0],
       [   1],
       [   2],
       ...,
       [9997],
       [9998],
       [9999]], dtype=uint16)

In [37]:
from maxentropy.utils import evaluate_feature_matrix

In [38]:
F = evaluate_feature_matrix(features, samplespace, matrix_format='ndarray')

In [39]:
F.shape

(10000, 9)

In [40]:
# %timeit evaluate_feature_matrix(features, samplespace, matrix_format='ndarray')

#### Performance oddity with range() and numpy integers

In [41]:
# %timeit samplespace[1000] in range(1000)   # super slow!! with a 16-bit uint dtype

In [42]:
s = set(range(1000))

In [43]:
# %timeit samplespace[1000] in s             # much faster

In [44]:
thing = np.array(1000, dtype=int)

In [45]:
# %timeit thing in range(1000)               # also very slow

In [46]:
# %timeit 1000 in range(1000)                # even this is slower than a set. Poor implementation of range.__contains__!

### Back to the model

In [47]:
samplespace.reshape(-1, 1)

array([[   0],
       [   1],
       [   2],
       ...,
       [9997],
       [9998],
       [9999]], dtype=uint16)

In [48]:
F = transformer.transform(samplespace)  # .reshape(-1, 1))

In [49]:
# %timeit F = transformer.transform(samplespace.reshape(-1, 1))

In [50]:
F

<10000x9 sparse matrix of type '<class 'numpy.float64'>'
	with 15298 stored elements in Compressed Sparse Column format>

In [51]:
k = F.mean(axis=0)

In [52]:
k

matrix([[0.98  , 0.096 , 0.004 , 0.1   , 0.1   , 0.08  , 0.0798, 0.08  ,
         0.01  ]])

These are the empirical expectations over the entire sample space, weighting all postcodes from 0 to 9999 equally.

More likely, we would want to supply the desired feature expectations as constraints from **prior knowledge** and/or compute the expectations on an actual dataset.

Here is an example: suppose we know that:
- 

What prior information?

- Do all Australian postcodes have 4 digits? Yes.
- What range? 0200 to 9944
- What proportion of mail our system sees is for Australia versus overseas? 80% (assumed)
- 25% (assumed) of all mail goes to these 10 postcodes: 2000, 2001, 3000, 3001, 4000, 4001, 5000, 5001, 6000, 6001.
- What population for each postcode? (and how to handle this when e.g. Paramatta NSW has 2150 for street addresses and 2124 for postcodes). If we don't have population info by postcode, how about using a prior with state population data?
   - NSW: postcodes 1000-1999 (PO boxes), 2000-2599, 2620-2899, 2921-2999
   - ACT: 0200-0299 (PO boxes), 2600-2619, 2900-2920
   - VIC: 3000-3999, 8000-8999 (PO boxes)
   - QLD: 4000-4999, 9000-9999 (PO boxes)
   - SA: 5000-5799, 5800-5999 (PO boxes)
   - WA 6000-6797, 6800-6999 (PO boxes)
   - TAS: 7000-7799, 7800-7999 (PO boxes)
   - NT: 0800-0899, 0900-0999 (PO boxes)

In [59]:
state_populations

,Population,%
State,,
New South Wales,7757843,32.0
Victoria,6100877,25.2
Queensland,4860448,20.1
South Australia,1710804,7.1
Western Australia,2623164,10.8
Tasmania,519783,2.1
Northern Territory,245657,1.0
Australian Capital Territory,398349,1.6


In [27]:
features = [is_valid,
            is_nsw_street, is_act_street, is_vic_street, is_qld_street,
            is_sa_street, is_wa_street, is_tas_street, is_nt_street]

In [70]:
k = np.array([[0.95,
               0.3, 0.02, 0.25, 0.2,
               0.07, 0.11, 0.02, 0.01]])
k /= (k.sum() - k[0, 0])

In [71]:
k.shape

(1, 9)

In [75]:
k[0, 1:].sum()

1.0

In [77]:
model = MinDivergenceModel(features, samplespace, matrix_format='ndarray', verbose=False)

In [78]:
model

MinDivergenceModel(feature_functions=[<function is_valid at 0x13bc019e0>,
                                      <function is_nsw_street at 0x13bc01da0>,
                                      <function is_act_street at 0x13bc02020>,
                                      <function is_vic_street at 0x13bc02480>,
                                      <function is_qld_street at 0x13bc02840>,
                                      <function is_sa_street at 0x13bc02de0>,
                                      <function is_wa_street at 0x13bc031a0>,
                                      <function is_tas_street at 0x13bc03560>,
                                      <function is_nt_street at 0x13bc037e0>],
                   matrix_format='ndarray',
                   samplespace=array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=uint16),
                   verbose=False)

In [79]:
k.shape

(1, 9)

In [80]:
model.fit(k)

DivergenceError: "dual is below the threshold 'mindual' and may be diverging to -inf.  Fix the constraints or lower the threshold!"

In [ ]:
%debug

In [ ]:
model.params_

In [ ]:
len(features)

In [ ]:
samplespace.shape

In [ ]:
k.shape

In [ ]:
model.model.F.shape

In [ ]:
model.model.resetparams()

In [ ]:
model.model.params

In [ ]:
from maxentropy import model

In [ ]:
maxentmodel = model.Model(features, samplespace, format='ndarray')

In [ ]:
k.shape

In [ ]:
maxentmodel.F

In [ ]:
maxentmodel.F.mean()

In [ ]:
maxentmodel.F.shape

In [ ]:
maxentmodel.resetparams()

In [ ]:
maxentmodel.params

In [ ]:
maxentmodel.log_norm_constant()

In [ ]:
maxentmodel.expectations()

In [ ]:
k

In [ ]:
maxentmodel.probdist()

So each postcode on the sample space has a probability $10^{-4}$ under our unfitted prior model.

In [ ]:
maxentmodel.showdist()

In [ ]:
maxentmodel.fit(np.squeeze(k))

In [ ]:
maxentmodel.params

In [ ]:
maxentmodel.feature_expectations()

In [ ]:
maxentmodel.params

In [ ]:
maxentmodel.probdist()